## Создаем подключение к базе данных с помощью библиотеки sqlite3

In [77]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('../data/checking-logs.sqlite')
cursor = conn.cursor()

## Получить схему таблицы test

In [78]:
cursor.execute("PRAGMA table_info(test);")
schema = cursor.fetchall()
for column in schema:
    print(column)

(0, 'uid', 'TEXT', 0, None, 0)
(1, 'labname', 'TEXT', 0, None, 0)
(2, 'first_commit_ts', 'TIMESTAMP', 0, None, 0)
(3, 'first_view_ts', 'TIMESTAMP', 0, None, 0)


## Получить только первые 10 строк таблицы test, чтобы проверить, как выглядит таблица

In [79]:
cursor.execute("SELECT * FROM test LIMIT 10;")
first_10_rows = cursor.fetchall()
for row in first_10_rows:
    print(row)

('user_1', 'laba04', '2020-04-26 17:06:18.462708', '2020-04-26 21:53:59.624136')
('user_1', 'laba04s', '2020-04-26 17:12:11.843671', '2020-04-26 21:53:59.624136')
('user_1', 'laba05', '2020-05-02 19:15:18.540185', '2020-04-26 21:53:59.624136')
('user_1', 'laba06', '2020-05-17 16:26:35.268534', '2020-04-26 21:53:59.624136')
('user_1', 'laba06s', '2020-05-20 12:23:37.289724', '2020-04-26 21:53:59.624136')
('user_1', 'project1', '2020-05-14 20:56:08.898880', '2020-04-26 21:53:59.624136')
('user_10', 'laba04', '2020-04-25 08:24:52.696624', '2020-04-18 12:19:50.182714')
('user_10', 'laba04s', '2020-04-25 08:37:54.604222', '2020-04-18 12:19:50.182714')
('user_10', 'laba05', '2020-05-01 19:27:26.063245', '2020-04-18 12:19:50.182714')
('user_10', 'laba06', '2020-05-19 11:39:28.885637', '2020-04-18 12:19:50.182714')


## Найти среди всех пользователей минимальное значение дельты между первым коммитом пользователя и крайним сроком соответствующей лабораторной работы, используя только один запрос
- сделайте это, присоединившись к таблице с deadlines
- разница должна отображаться в часах
- не принимайте во внимание лабораторную работу «проект 1», у нее более длительные сроки выполнения, и она будет выделяться
- значение должно быть сохранено в фрейме данных df_min с соответствующим uid

--------------------------------------------------------------------------------------------------------------------------------------------------------

In [80]:
# Получение схемы таблицы deadlines
cursor.execute("PRAGMA table_info(deadlines);")
schema = cursor.fetchall()
for column in schema:
    print(column)

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'labs', 'TEXT', 0, None, 0)
(2, 'deadlines', 'INTEGER', 0, None, 0)


In [81]:
# Получение схемы таблицы test
cursor.execute("PRAGMA table_info(test);")
schema = cursor.fetchall()
for column in schema:
    print(column)

(0, 'uid', 'TEXT', 0, None, 0)
(1, 'labname', 'TEXT', 0, None, 0)
(2, 'first_commit_ts', 'TIMESTAMP', 0, None, 0)
(3, 'first_view_ts', 'TIMESTAMP', 0, None, 0)


In [82]:
# Проверка формата данных в столбце deadlines.deadlines
cursor.execute("SELECT deadlines FROM deadlines LIMIT 10;")
deadlines_sample = cursor.fetchall()
print("Sample deadlines:", deadlines_sample)

# Проверка формата данных в столбце test.first_commit_ts
cursor.execute("SELECT first_commit_ts FROM test LIMIT 10;")
first_commit_sample = cursor.fetchall()
print("Sample first_commit_ts:", first_commit_sample)

Sample deadlines: [(1587945599,), (1587945599,), (1588550399,), (1590364799,), (1590364799,), (1589673599,)]
Sample first_commit_ts: [('2020-04-26 17:06:18.462708',), ('2020-04-26 17:12:11.843671',), ('2020-05-02 19:15:18.540185',), ('2020-05-17 16:26:35.268534',), ('2020-05-20 12:23:37.289724',), ('2020-05-14 20:56:08.898880',), ('2020-04-25 08:24:52.696624',), ('2020-04-25 08:37:54.604222',), ('2020-05-01 19:27:26.063245',), ('2020-05-19 11:39:28.885637',)]


<i>Тут видим, что столбец deadlines.deadlines содержит значения в формате UNIX-времени (timestamp), а столбец test.first_commit_ts содержит значения в формате строки даты и времени.
С помощью функции datetime мы потом преобразуем UNIX-время в формат даты и времени</i>

--------------------------------------------------------------------------------------------------------------------------------------------------------

In [83]:
query_min = """
SELECT uid, MIN(min_diff) AS global_min_diff
FROM (
    SELECT test.uid, MIN((julianday(test.first_commit_ts) - julianday(datetime(deadlines.deadlines, 'unixepoch'))) * 24) AS min_diff
    FROM test
    JOIN deadlines ON test.labname = deadlines.labs
    WHERE deadlines.labs != 'project1'
      AND test.first_commit_ts IS NOT NULL
      AND deadlines.deadlines IS NOT NULL
    GROUP BY test.uid
) AS subquery;
"""
cursor.execute(query_min)
result_min = cursor.fetchone()
df_min = pd.DataFrame([result_min], columns=['uid', 'global_min_diff'])
print("Минимальное значение min_diff:")
print(df_min)

Минимальное значение min_diff:
       uid  global_min_diff
0  user_30       -202.38473


## Сделайте то же самое, но для максимума, используя только один запрос, имя dataframe будет df_max

In [84]:
query_max = """
SELECT uid, MAX(max_diff) AS global_max_diff
FROM (
    SELECT test.uid, MAX((julianday(test.first_commit_ts) - julianday(datetime(deadlines.deadlines, 'unixepoch'))) * 24) AS max_diff
    FROM test
    JOIN deadlines ON test.labname = deadlines.labs
    WHERE deadlines.labs != 'project1'
      AND test.first_commit_ts IS NOT NULL
      AND deadlines.deadlines IS NOT NULL
    GROUP BY test.uid
) AS subquery;
"""
cursor.execute(query_max)
result_max = cursor.fetchone()
df_max = pd.DataFrame([result_max], columns=['uid', 'global_max_diff'])
print("Максимальное значение max_diff:")
print(df_max)

Максимальное значение max_diff:
       uid  global_max_diff
0  user_25        -2.867236


## Сделайте то же самое, но для среднего значения, используя только один запрос, на этот раз ваш фрейм данных не должен включать столбец uid, а имя фрейма данных — df_avg

In [85]:
query = """
SELECT AVG((julianday(test.first_commit_ts) - julianday(datetime(deadlines.deadlines, 'unixepoch'))) * 24) AS avg_diff
FROM test
JOIN deadlines ON test.labname = deadlines.labs
WHERE deadlines.labs != 'project1'
  AND test.first_commit_ts IS NOT NULL
  AND deadlines.deadlines IS NOT NULL;
"""
cursor.execute(query)
result = cursor.fetchone()
df_avg = pd.DataFrame([result], columns=['avg_diff'])
print("Средняя дельта (avg_diff):")
print(df_avg)

Средняя дельта (avg_diff):
    avg_diff
0 -89.687686


## Мы хотим проверить гипотезу о том, что пользователи, которые посетили новостную ленту всего несколько раз, имеют более низкую дельту между первым коммитом и дедлайном. Для этого нужно рассчитать коэффициент корреляции между количеством просмотров страниц и разницей
- Используя только один запрос, создайте таблицу со столбцами: uid, avg_diff, pageviews
- uid — это uid, которые существуют в тесте
- avg_diff — это средняя разница между первым коммитом и крайним сроком выполнения лабораторных работ на пользователя.
- просмотры страниц — это количество посещений новостной ленты на одного пользователя
- не принимайте во внимание лабораторную работу «проект1»
- сохраните его в dataframe views_diff
- используйте метод Pandas corr() для расчета коэффициента корреляции между количеством просмотров страниц и разницей

In [86]:
query = """
SELECT
    test.uid,
    AVG((julianday(datetime(deadlines.deadlines, 'unixepoch')) - julianday(test.first_commit_ts)) * 24) AS avg_diff,
    COUNT(test.first_view_ts) AS pageviews
FROM
    test
JOIN
    deadlines ON test.labname = deadlines.labs
WHERE
    deadlines.labs != 'project1'
    AND test.first_commit_ts IS NOT NULL
    AND deadlines.deadlines IS NOT NULL
    AND test.uid LIKE 'user_%'
GROUP BY
    test.uid;
"""
views_diff = pd.read_sql_query(query, conn)

# Вывод таблицы с uid, avg_diff и pageviews
print("Таблица с uid, avg_diff и pageviews:")
print(views_diff)

# Расчет коэффициента корреляции между avg_diff и pageviews
correlation = views_diff[['avg_diff', 'pageviews']].corr()
print("Коэффициент корреляции между avg_diff и pageviews:")
print(correlation)

Таблица с uid, avg_diff и pageviews:
        uid    avg_diff  pageviews
0    user_1   65.119644          5
1   user_10   75.242310          5
2   user_14  159.568696          3
3   user_17   62.207514          5
4   user_18    6.367907          3
5   user_19   99.440298          4
6   user_21   96.111042          4
7   user_25   93.474751          5
8   user_28   86.793652          5
9    user_3  105.738041          5
10  user_30  145.528546          4
Коэффициент корреляции между avg_diff и pageviews:
           avg_diff  pageviews
avg_diff   1.000000  -0.118374
pageviews -0.118374   1.000000


## Закрыть соединение

In [87]:
conn.close()